In [88]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
#from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
import os

In [12]:
#load data and turn to lowercase
filename = 'data.txt'
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text =raw_text.lower() #reduce no of unique characcters

In [13]:
#read first 300 characters
raw_text[0:300]

"project gutenberg's alice's adventures in wonderland, by lewis carroll\n\nthis ebook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  you may copy it, give it away or\nre-use it under the terms of the project gutenberg license included\nwith this ebook or online "

In [17]:
#now these characters cant unders stand bn nural net we hv to conver characters to numbers

#1st find unique charcters in text ==>  we can use set to find unique charcterss it remove duplicates when creating a set
#then put set of charcters to list and sorted in order

char  = sorted(list(set(raw_text)))

#now map charcter to integer
#enmurate ==> give number with character 
#0 h
#1 e
#2 l
#3 l
#4 o

#using dict we pair them

chars_to_int = dict((c, i) for i, c in enumerate(char))

In [19]:
#character in the text withing dict
print(chars_to_int)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, "'": 7, '(': 8, ')': 9, '*': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '?': 27, '@': 28, '[': 29, ']': 30, '_': 31, 'a': 32, 'b': 33, 'c': 34, 'd': 35, 'e': 36, 'f': 37, 'g': 38, 'h': 39, 'i': 40, 'j': 41, 'k': 42, 'l': 43, 'm': 44, 'n': 45, 'o': 46, 'p': 47, 'q': 48, 'r': 49, 's': 50, 't': 51, 'u': 52, 'v': 53, 'w': 54, 'x': 55, 'y': 56, 'z': 57}


In [21]:
#no of charcters in text file
n_chars = len(raw_text)
print("total charcters : " , n_chars)

total charcters :  163780


In [23]:
#no of unique charcters in text file(vocabulary)
n_vocab = len(char)
print("total charcters : " , n_vocab)

total charcters :  58


In [25]:
#Prepare the dataset of input to output pairs encoded as integers.
#now we need x and y data to train and test model
#so we take range of charcater as x and nexr charcater as y
#chapt --> e  
#hapte --> r

#next we take the integer related to charcter usin char_to_int dictionary and put it into varible lists


In [27]:
seq_length = 100 #range of data we take as x in sequentialy
dataX = [] #independet(x) text pattern
dataY = [] #next charcter of the pattern

#loop to create X & Y and put to lists
#n_chars - seq_length ==> use this bucuse at the end of last 100 charcters text is over
for i in range (0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i+seq_length]   #get sequence of charcters withing range of 100
    seq_out = raw_text[i+seq_length]    #get next character
    dataX.append([chars_to_int[char] for char in seq_in])    #append all the charcters in pattern using char_to_int dict as a list to the patterns list
    dataY.append(chars_to_int[seq_out])      #append next charcter in the pattern to list

n_patterns = len(dataX)   #no of patterns with 100 charcters in dequce in the text

print('total patterns : ', n_patterns)


total patterns :  163680


In [29]:
#checking random pattern with next charccter
print('pattern : ', dataX[163679])
print('next character : ', dataY[163679])

pattern :  [1, 39, 36, 43, 47, 1, 47, 49, 46, 35, 52, 34, 36, 1, 46, 52, 49, 1, 45, 36, 54, 1, 36, 33, 46, 46, 42, 50, 11, 1, 32, 45, 35, 1, 39, 46, 54, 1, 51, 46, 0, 50, 52, 33, 50, 34, 49, 40, 33, 36, 1, 51, 46, 1, 46, 52, 49, 1, 36, 44, 32, 40, 43, 1, 45, 36, 54, 50, 43, 36, 51, 51, 36, 49, 1, 51, 46, 1, 39, 36, 32, 49, 1, 32, 33, 46, 52, 51, 1, 45, 36, 54, 1, 36, 33, 46, 46, 42, 50, 13]
next character :  0


In [35]:
#Transform the list of input sequences into the form [samples, time steps, features] that is expected by an LSTM
# rescale the integers to the range [0,1] to make the patterns easier to learn by the LSTM network
# uses the sigmoid activation function by default.

X = np.reshape(dataX, (n_patterns, seq_length, 1))  #we reshape dataX in form of [samples, time steps, features]

X = X/ float(n_vocab)  #rescale to 0 -1

print(X)

[[[0.81034483]
  [0.84482759]
  [0.79310345]
  ...
  [0.01724138]
  [0.79310345]
  [0.63793103]]

 [[0.84482759]
  [0.79310345]
  [0.70689655]
  ...
  [0.79310345]
  [0.63793103]
  [0.01724138]]

 [[0.79310345]
  [0.70689655]
  [0.62068966]
  ...
  [0.63793103]
  [0.01724138]
  [0.55172414]]

 ...

 [[0.87931034]
  [0.79310345]
  [0.01724138]
  ...
  [0.79310345]
  [0.79310345]
  [0.72413793]]

 [[0.79310345]
  [0.01724138]
  [0.67241379]
  ...
  [0.79310345]
  [0.72413793]
  [0.86206897]]

 [[0.01724138]
  [0.67241379]
  [0.62068966]
  ...
  [0.72413793]
  [0.86206897]
  [0.22413793]]]


In [43]:
#Convert the output values (single characters converted to integers) into a one hot encoding.
#one hot encoding == > 31 ==> 00000000......100000.000 => 31st intest is 1 & rest 57 is Zero

Y = tf.keras.utils.to_categorical(dataY)

print(Y.shape)

(163680, 58)


In [53]:
#build model
model = Sequential()
model.add(LSTM(255, input_shape = (X.shape[1], X.shape[2]))) 
#255--> no of neurons in LSTM laayer
#X.shape[1] --> no of time steps
#X.shape[12] --> no of features
model.add(Dropout(0.2)) #20% of neurons drop when training to avoid overfitting
model.add(Dense(Y.shape[1], activation = 'softmax')) #Y.shape[1] = 58 no of outputs

In [55]:
#compile
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam'
)

In [70]:
#define checkpoint ==> to save weight values to use to new model when creating
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.keras"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callback_list = [checkpoint]

In [74]:
epochs = 10
batch_size =128 #no of samples proccesedd befor entire set of weights are changed

In [76]:
#moedl fit
model.fit(X, Y, epochs = epochs, batch_size=batch_size, callbacks=callback_list)

Epoch 1/10
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 3.0724
Epoch 1: loss improved from inf to 2.97360, saving model to weights-improvement-01-2.9736.keras
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 187s 144ms/step - loss: 3.0723
Epoch 2/10
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - loss: 2.8196
Epoch 2: loss improved from 2.97360 to 2.79733, saving model to weights-improvement-02-2.7973.keras
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 177s 138ms/step - loss: 2.8196
Epoch 3/10
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - loss: 2.7302
Epoch 3: loss improved from 2.79733 to 2.70455, saving model to weights-improvement-03-2.7046.keras
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 190s 148ms/step - loss: 2.7302
Epoch 4/10
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - loss: 2.6419
Epoch 4: loss improved from 2.70455 to 2.63354, saving model to weights-improvement-04-2.6335.keras
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 194s 151ms/step - loss: 2.6419
Epoch 5/10
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - loss: 2.5858
E

In [100]:
#Generate Text with the trained LSTM model
filename = "C:\\Users\\damin\\Downloads\\Practical Materials - Lab 6\\Practical Materials - Lab 6"
model.save_weights(os.path.join(filename, "weights-improvement-10-2.3431.weights.h5"))
model.compile(loss='categorical_cressentropy', optimizer='adam')

In [104]:
int_to_char = dict((i, c) for i, c in enumerate(char))

In [146]:
#genarate random seed sequence
start = np.random.randint(0, len(dataX)-1)
print(start)

118134


In [148]:
#pattern of the random list position
pattern = dataX[start]
print(pattern)

[45, 1, 50, 39, 36, 1, 37, 40, 49, 50, 51, 0, 50, 32, 54, 1, 51, 39, 36, 1, 54, 39, 40, 51, 36, 1, 49, 32, 33, 33, 40, 51, 13, 1, 50, 39, 36, 1, 54, 32, 50, 1, 32, 1, 43, 40, 51, 51, 43, 36, 1, 45, 36, 49, 53, 46, 52, 50, 1, 32, 33, 46, 52, 51, 1, 40, 51, 1, 41, 52, 50, 51, 1, 32, 51, 1, 37, 40, 49, 50, 51, 11, 0, 51, 39, 36, 1, 51, 54, 46, 1, 34, 49, 36, 32, 51, 52, 49, 36, 50]


In [150]:
#turn int to characters
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

" n she first
saw the white rabbit. she was a little nervous about it just at first,
the two creatures "


In [164]:
#genarate characteres
length = 0 #no of charcters want to genrate
final = []
for i in range(length):
    #reshape sequce before sending to LSTM
    x = np.reshape(pattern, (1, len(pattern), 1))
    #normalize values
    x = x/float(n_vocab)
    #predictions
    predictions = model.predict(x, verbose =0)
    index = np.argmax(predictions)
    result = int_to_char[index]
    final.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print(final)

[]
